In [1]:
import numpy as np
import os, cv2
from mxnet import gluon, nd, image
from mxnet.gluon import data as gdata, utils as gutils
from shutil import copyfile
from mxnet.gluon.data.vision import datasets, transforms

/home/lab/anaconda3/envs/mxnet-gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
target_folder = './Fine_Tuning_Data_2_Origin/'
output_folder = './Fine_Tuning_Data_2/'

### Training Data Augmentaion

In [3]:
files = os.listdir(target_folder)
pictures_resized = []
for f in files:
    if '.jpg' in f:
        pictures_resized.append(f)

for index in range(len(pictures_resized)):
#for index in range(1):
    
    name = pictures_resized[index]
    img = image.imread(target_folder+name)
    img_np = img.asnumpy()
    num_jitter = 2
    #num_jitter = 5
    color_aug = gdata.vision.transforms.RandomColorJitter(brightness=0.4, contrast=0.3)
    #color_aug = gdata.vision.transforms.RandomColorJitter(brightness=0.25, contrast=0.6)
    aug_1 = gdata.vision.transforms.RandomFlipLeftRight()
    aug_2 = gdata.vision.transforms.RandomFlipTopBottom()

# left-right flip
    success = False
    while not success:
        img_lr_flip = aug_1(img)
        img_lr_flip = img_lr_flip.asnumpy()
        if (img_lr_flip!=img_np).sum()!=0:
            success = True
            cv2.imwrite(output_folder+name.split('.')[0]+'_left_right_flipped.jpg', img_lr_flip[:,:,::-1])
# top-down flip
    success = False
    while not success:
        img_tb_flip = aug_2(img)
        img_tb_flip = img_tb_flip.asnumpy()
        if (img_tb_flip!=img_np).sum()!=0:
            success = True
            cv2.imwrite(output_folder+name.split('.')[0]+'_top_down_flipped.jpg', img_tb_flip[:,:,::-1])
# color jitters
    for i in range(num_jitter):
        name_jitter = str('_color_jitter%d'%(i))
        img_color_jitter = color_aug(img)
        img_color_jitter_np = img_color_jitter.asnumpy()
        cv2.imwrite(output_folder+name.split('.')[0]+name_jitter+'.jpg', img_color_jitter_np[:,:,::-1])
# left-right flip-color jitters        
        success = False
        while not success:
            img_lr_flip = aug_1(img_color_jitter)
            img_lr_flip = img_lr_flip.asnumpy()
            if (img_lr_flip!=img_color_jitter_np).sum()!=0:
                success = True
                cv2.imwrite(output_folder+name.split('.')[0]+name_jitter+name+'_left_right_flipped.jpg', img_lr_flip[:,:,::-1])
# top-down flip-color jitters
        success = False
        while not success:
            img_tb_flip = aug_2(img_color_jitter)
            img_tb_flip = img_tb_flip.asnumpy()
            if (img_tb_flip!=img_color_jitter_np).sum()!=0:
                success = True
                cv2.imwrite(output_folder+name.split('.')[0]+name_jitter+name+'_top_down_flipped.jpg', img_tb_flip[:,:,::-1])
    copyfile(target_folder+name, output_folder+name)

files = os.listdir(output_folder)
pictures_list = []
for f in files:
    if '.jpg' in f:
        pictures_list.append(f)
        
for name in pictures_list:
    img = image.imread(output_folder+name)
    img_np = img.asnumpy()
    
    img_transpose = np.zeros_like(img_np)
    for channels in range(3):
        img_transpose[:,:,channels] = img_np[:,:, channels].T
    cv2.imwrite(output_folder+name.split('.')[0]+'_transposed.jpg', img_transpose[:,:,::-1])